In [21]:
import os
import pandas as pd
import re

def clean_text(text):
    # Remove special characters from the text
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    return cleaned_text

def match_line(line, df):
    cleaned_line = clean_text(line.lower())  # Clean the line by removing special characters and converting to lowercase
    matched_rows = []  # Create a list to store the matched rows
    for _, row in df.iterrows():
        transcription = clean_text(str(row['TRANSCRIPTION (SESOTHO)']).lower())
        translation = clean_text(str(row['TRANSLATION (ENGLISH)']).lower())
        # print(translation,line,'Line')
        if cleaned_line in transcription or cleaned_line in translation:
            matched_rows.append(row)  # Add the matched row to the list
    # Convert the list of matched rows to a DataFrame
    matched_rows_df = pd.concat(matched_rows, axis=1).transpose() if matched_rows else pd.DataFrame()
    return matched_rows_df


folder_path = "../Interview XLSX/"
file_extension = ".xlsx"

combined_df = pd.DataFrame()

for filename in os.listdir(folder_path):
    if filename.endswith(file_extension):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_excel(file_path)
        df['filepath'] = file_path
        combined_df = pd.concat([combined_df, df], ignore_index=True)

matched_rows = []

with open('../Script/test script.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

narrator = None
language = None
for line in lines:
    line = line.strip().lower()
    if line.startswith('***'):
        narrator = line[3:].strip().lower()
        language = 'SESOTHO'
    elif line.startswith('###'):
        narrator = line[3:].strip().lower()
        language = 'ENGLISH'
    elif line and narrator and language:
        matched_rows_df = match_line(line, combined_df)
        print(matched_rows_df)
        if not matched_rows_df.empty:
            for _, row in matched_rows_df.iterrows():
                new_row = {
                    'Text': line,
                    'Narrator': narrator,
                    'Language': language,
                    'Timecode Range': row['TIME'],
                    'FilePath': row['filepath']
                }
                matched_rows.append(new_row)
        else:  # If no match found, save line with no timecode range or filepath
            new_row = {
                'Text': line,
                'Narrator': narrator,
                'Language': language,
                'Timecode Range': None,
                'FilePath': None
            }
            matched_rows.append(new_row)

matched_df = pd.DataFrame(matched_rows)
# print(matched_df)
matched_df.to_csv('../intermedia.csv', index=False)


           TIME                            TRANSCRIPTION (SESOTHO)  \
17  13:33-13:54  Ntho eo ke ratang ho e etsa le Moselantja ke h...   

                                TRANSLATION (ENGLISH)  \
17  What I would like to do with Moselantja is for...   

                                    filepath  
17  ../Interview XLSX/Grandma Interview.xlsx  
           TIME                            TRANSCRIPTION (SESOTHO)  \
17  13:33-13:54  Ntho eo ke ratang ho e etsa le Moselantja ke h...   

                                TRANSLATION (ENGLISH)  \
17  What I would like to do with Moselantja is for...   

                                    filepath  
17  ../Interview XLSX/Grandma Interview.xlsx  
           TIME                            TRANSCRIPTION (SESOTHO)  \
18  14:04-14:20  Khotso e bolela bophelo bo bottle ka hare ho l...   

                           TRANSLATION (ENGLISH)  \
18  Peace means peaceful life within the family.   

                                    filepath  
18  ../

In [ ]:
# that two of the interviewees dont have xlsx, only txt files. 
# An additional simple python script will be needed to make this conversion
# in case the python script can identify all of these lines correctly, I recommend you add an additional 
# incorrect line to the script, to simulate the case where it doesn't identify it right, to make sure we are handling that

import csv
import os
import xml.etree.ElementTree as ET
import copy


def convert_timecode(timecode):
    # Convert timecode from 'HH:MM-SS:MM' format to 'HH:MM:SS:FF' format
    start_time, end_time = timecode.split('-')
    start_time_parts = start_time.split(':')
    end_time_parts = end_time.split(':')

    # Convert minutes to seconds and add frames
    start_seconds = int(start_time_parts[0]) * 60 + int(start_time_parts[1])
    end_seconds = int(end_time_parts[0]) * 60 + int(end_time_parts[1])

    # Format as 'HH:MM:SS:FF'
    converted_start_time = '{:02d}:{:02d}:00:00'.format(start_seconds // 60, start_seconds % 60)
    converted_end_time = '{:02d}:{:02d}:00:00'.format(end_seconds // 60, end_seconds % 60)

    return converted_start_time, converted_end_time

def is_time_within_range(time, start_time, end_time):
    return start_time <= time <= end_time

def get_parent_map(root):
    parent_map = {c: p for p in root.iter() for c in p}
    return parent_map

def get_parent_element(parent_map, element):
    return parent_map.get(element)

def get_clip_type(parent_map, element):
    parent = get_parent_element(parent_map, element)
    while parent is not None:
        if parent.tag in ['video', 'audio']:
            return parent.tag
        parent = get_parent_element(parent_map, parent)
    return 'unknown'


def create_xml_structure(project_name, matches):
    # Create the root element and set attributes
    root = ET.Element("xmeml")
    root.set("version", "4")
    
    # Create the project element
    project = ET.SubElement(root, "project")
    
    # Create the name element and set the project name
    name = ET.SubElement(project, "name")
    name.text = project_name
    
    # Create the children element
    children = ET.SubElement(project, "children")
    
    # Create the sequence element
    sequence = ET.SubElement(children, "sequence")
    sequence.set("id", "sequence-1")
    sequence.set("TL.SQAudioVisibleBase", "0")
    sequence.set("TL.SQVideoVisibleBase", "0")
    sequence.set("TL.SQVisibleBaseTime", "1954072810692088")
    sequence.set("TL.SQAVDividerPosition", "0.556213021278")
    sequence.set("MZ.Sequence.PreviewUseMaxRenderQuality", "false")
    sequence.set("MZ.Sequence.PreviewUseMaxBitDepth", "false")
    sequence.set("MZ.Sequence.VideoTimeDisplayFormat", "998")
    sequence.set("MZ.WorkOutPoint", "8014566084840000")
    sequence.set("MZ.WorkInPoint", "0")
    sequence.set("explodedTracks", "true")
    
    # Create the duration element and set a value
    duration = ET.SubElement(sequence, "duration")
    duration.text = "424764"
    
    # Create the rate element and set the timebase and ntsc values
    rate = ET.SubElement(sequence, "rate")
    
    timebase = ET.SubElement(rate, "timebase")
    timebase.text = "53"
    
    ntsc = ET.SubElement(rate, "ntsc")
    ntsc.text = "FALSE"
    
    # Create the name element and set a value
    sequence_name = ET.SubElement(sequence, "name")
    sequence_name.text = "output"
    
    # Create the media element
    media = ET.SubElement(sequence, "media")
    
    # Create the video element
    video = ET.SubElement(media, "video")
    video_tracks = {}  # Store video tracks

    # Create the audio element
    audio = ET.SubElement(media, "audio")
    audio_tracks = {}  # Store audio tracks

    video_clips = [clip for track, clips in matches.items() for clip, ctype, _ in clips if ctype == "video"]
    audio_clips = [clip for track, clips in matches.items() for clip, ctype, _ in clips if ctype == "audio"]

    video_tracks_num = 2 if len(video_clips) <= 10 else 3
    audio_tracks_num = 2 if len(audio_clips) <= 10 else 3

    video_clips_per_track = [video_clips[i::video_tracks_num] for i in range(video_tracks_num)]
    audio_clips_per_track = [audio_clips[i::audio_tracks_num] for i in range(audio_tracks_num)]

    for track_index in range(video_tracks_num):
        video_track = ET.SubElement(video, "track")
        video_tracks[track_index] = video_track  # Add new track to dict
        for clip_item in video_clips_per_track[track_index]:
            video_tracks[track_index].append(copy.deepcopy(clip_item))  # Append clip to track

    for track_index in range(audio_tracks_num):
        audio_track = ET.SubElement(audio, "track")
        audio_tracks[track_index] = audio_track  # Add new track to dict
        for clip_item in audio_clips_per_track[track_index]:
            audio_tracks[track_index].append(copy.deepcopy(clip_item))  # Append clip to track


    # Create the timecode element
    timecode = ET.SubElement(sequence, "timecode")
    
    # Create the rate element for timecode and set the timebase and ntsc values
    timecode_rate = ET.SubElement(timecode, "rate")
    
    timecode_timebase = ET.SubElement(timecode_rate, "timebase")
    timecode_timebase.text = "53"
    
    timecode_ntsc = ET.SubElement(timecode_rate, "ntsc")
    timecode_ntsc.text = "FALSE"
    
    # Create the string element for timecode and set a value
    timecode_string = ET.SubElement(timecode, "string")
    timecode_string.text = "00:00:00:00"
    
    # Create the frame element and set a value
    frame = ET.SubElement(timecode, "frame")
    frame.text = "0"
    
    # Create the displayformat element and set a value
    display_format = ET.SubElement(timecode, "displayformat")
    display_format.text = "NDF"
    
    # Create the labels element
    labels = ET.SubElement(sequence, "labels")
    
    # Create the label2 element and set a value
    label2 = ET.SubElement(labels, "label2")
    label2.text = "Forest"
    
    # Create the logginginfo element
    logging_info = ET.SubElement(sequence, "logginginfo")
    
    # Create the child elements for logginginfo and leave them empty
    child_elements = ["description", "scene", "shottake", "lognote", "good", "originalvideofilename", "originalaudiofilename"]
    for element in child_elements:
        ET.SubElement(logging_info, element)
    
    # Create the XML tree
    tree = ET.ElementTree(root)
     
    # Return the XML tree
    return tree


def extract_clips(csv_file, xml_folder, output_file):
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        matches = {}

        for row in reader:
            timecode_range = row['Timecode Range']
            if timecode_range and timecode_range != 'None':
                start_time, end_time = convert_timecode(timecode_range)

                xml_files = os.listdir(xml_folder)
                for xml_file in xml_files:
                    xml_path = os.path.join(xml_folder, xml_file)
                    tree = ET.parse(xml_path)
                    root = tree.getroot()
                    parent_map = get_parent_map(root)  # create the parent map here
                    clip_items = root.findall(".//clipitem")

                    for clip_item in clip_items:
                        timecode_element = clip_item.find(".//timecode/string")
                        if timecode_element is not None:
                            xml_timecode = timecode_element.text

                            if is_time_within_range(xml_timecode, start_time, end_time):
                                clip_type = get_clip_type(parent_map, clip_item)  # get the clip type here

                                track_properties = {}
                                track_element = get_parent_element(parent_map, clip_item)
                                if track_element is not None and track_element.tag == 'track':
                                    for attribute in track_element.attrib:
                                        track_properties[attribute] = track_element.attrib[attribute]

                                print('Match found in {}'.format(xml_file))
                                print('Timecode: {}'.format(xml_timecode))
                                print('Start time: {}'.format(start_time))
                                print('End time: {}'.format(end_time))
                                print('Clip Type: {}'.format(clip_type))
                                print('Track Properties: {}'.format(track_properties))

                                # Add to matches dictionary
                                if track_element not in matches:
                                    matches[track_element] = []
                                matches[track_element].append((clip_item, clip_type, track_properties))

    # return matches

        if matches:
            output_tree = create_xml_structure('test', matches)
            output_tree.write(output_file, encoding='utf-8', xml_declaration=True)

# Usage example
csv_file = '../output.csv'
xml_folder = '../interview xmls'
output_file = '../xml exports//matched_clips.xml'

extract_clips(csv_file, xml_folder, output_file)


Match found in ross - synced.xml
Timecode: 13:50:18:00
Start time: 13:33:00:00
End time: 13:54:00:00
Clip Type: video
Track Properties: {}
Match found in ross - synced.xml
Timecode: 13:53:06:00
Start time: 13:33:00:00
End time: 13:54:00:00
Clip Type: video
Track Properties: {}
Match found in ross - synced.xml
Timecode: 13:50:18:00
Start time: 13:33:00:00
End time: 13:54:00:00
Clip Type: video
Track Properties: {}
Match found in ross - synced.xml
Timecode: 13:53:06:00
Start time: 13:33:00:00
End time: 13:54:00:00
Clip Type: video
Track Properties: {}
Match found in chrissy_love - synced.xml
Timecode: 14:10:40:00
Start time: 14:04:00:00
End time: 14:20:00:00
Clip Type: video
Track Properties: {}
Match found in mats'ele - synced.xml
Timecode: 14:09:17:00
Start time: 14:04:00:00
End time: 14:20:00:00
Clip Type: video
Track Properties: {}
Match found in ross - synced.xml
Timecode: 14:04:34:00
Start time: 14:04:00:00
End time: 14:20:00:00
Clip Type: video
Track Properties: {}
Match found in 